# Defining the click probability functions.
The goal of this notebook is define a function for the click probability. 
First it's defined the simplest expression, the probablility without including the factor $\eta$ and once it's known how to do it we define the complete function including $\eta$. 
As I'm learning Python first I have calculated how to do it without a function for being sure what I was doing and after I define the function.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.misc import factorial
%matplotlib inline

import math
import time


Define first the function that I want to fit. I'll use first the easy version:
 $$R(N)=1-e^{-N}\sum_{i=0}^m(1-p_i)\frac{N^i}{i!}$$
 First of all we'll calculate step by step what we want to do in the function to understand what we're doing and after we'll write it in a function.

In [3]:
#Let's calculate here step by step what we want to do in our function.
N = np.array([0,1, 2, 3, 4, 5, 6]) #This will be the number of photons
N_new = N[...,np.newaxis]
p = np.array([1, 10, 100]) #This will be the probabilities for each 
                           #detection. (Values don't matter now)
i = np.arange(len(p))

In [4]:
print(N_new**i)  #This result give us the in the first column the
#vector N to the power of the first element of the i array.
# In the second column gives the vector N to the power of the
#second element of i and so on

H = N_new**i/factorial(i) #Now each column is divided by the
#corresponding i factorial value. First column divided by the
#factorial of the first element of i
print(H)
multiplication = H*(1-p) #Same as for factorial(i)
print(multiplication)

Summation = np.sum(multiplication, axis = 1) #Here we sum the 
#elements of the same row because the belong to the same number 
#of N photons. As an output we obtain a vector that contains the
#summation for each value of N
print('Sumation')
print(Summation)


R = 1 - np.exp(-N) * Summation #The output is a vector that contais 
# the click probability for efficiency 1 for each value of N
print('R')
print(R)


[[ 1  0  0]
 [ 1  1  1]
 [ 1  2  4]
 [ 1  3  9]
 [ 1  4 16]
 [ 1  5 25]
 [ 1  6 36]]
[[  1.    0.    0. ]
 [  1.    1.    0.5]
 [  1.    2.    2. ]
 [  1.    3.    4.5]
 [  1.    4.    8. ]
 [  1.    5.   12.5]
 [  1.    6.   18. ]]
[[    0.     -0.     -0. ]
 [    0.     -9.    -49.5]
 [    0.    -18.   -198. ]
 [    0.    -27.   -445.5]
 [    0.    -36.   -792. ]
 [    0.    -45.  -1237.5]
 [    0.    -54.  -1782. ]]
Sumation
[    0.    -58.5  -216.   -472.5  -828.  -1282.5 -1836. ]
R
[  1.          22.52094731  30.23242118  24.5243898   16.165349
   9.64141703   5.550989  ]


Now we're ready to define the function:  $$R(N)=1-e^{N}\sum_{i=0}^m(1-p_i)\frac{N^i}{i!}$$

In [5]:
def Click1 (N, p):
#This function calculates the click probability for the simple
#case with efficiency equals to 1.
#Arguments: number of photons, N, and probability detection, p.
#In general both will be arrays.
    i = np.arange(len(p))
    N_new = N[..., np.newaxis]
    addend = (1-p) * N_new**i / factorial(i)  #This is a matrix that contains 
    # in each element one of the terms of the sumation. Each row corresponds
    # to the terms with the same number of N photons. Each column corresponds 
    # with the index of the summation.
    SUM = np.sum(addend, axis = 1) #each element of this vector is the sum for 
                              #each value of N
        
    # The returned value is a vector with the probabilities for each N
    return (1 - np.exp(-N) * SUM)


Now we'll create the complete function considering the efficiency $\eta$. 
$$R(N)=1-e^{-\eta N}\sum_{i=0}^m(1-p_i)\frac{(\eta N)^i}{i!}$$

In [6]:
#Let's calculate here step by step what we want to do in our function.
N = np.array([0, 1, 2, 3, 4, 5, 6]) #This will be the number of photons
eta = np.array([1, 2]) #This all the possible efficiencies
eta_new = eta[...,np.newaxis]
i = np.arange(len(p))
p = np.array([1, 10, 100]) #This will be the probabilities for each 
                           #detection. (Values don't matter now)



In [7]:
Neta = (N * eta_new)
#print(Neta)
Neta_new = Neta[...,np.newaxis]
addend = Neta_new**i / factorial(i) * (1-p)
print(addend)

SUM = np.sum(addend, axis = 2)
print(SUM)

#Now I have to multiply everything with the exponential that contais eta and N

R = 1-np.exp(-Neta)*SUM
print (R)

[[[    0.     -0.     -0. ]
  [    0.     -9.    -49.5]
  [    0.    -18.   -198. ]
  [    0.    -27.   -445.5]
  [    0.    -36.   -792. ]
  [    0.    -45.  -1237.5]
  [    0.    -54.  -1782. ]]

 [[    0.     -0.     -0. ]
  [    0.    -18.   -198. ]
  [    0.    -36.   -792. ]
  [    0.    -54.  -1782. ]
  [    0.    -72.  -3168. ]
  [    0.    -90.  -4950. ]
  [    0.   -108.  -7128. ]]]
[[    0.    -58.5  -216.   -472.5  -828.  -1282.5 -1836. ]
 [    0.   -216.   -828.  -1836.  -3240.  -5040.  -7236. ]]
[[  1.          22.52094731  30.23242118  24.5243898   16.165349
    9.64141703   5.550989  ]
 [  1.          30.23242118  16.165349     5.550989     2.08689891
    1.22881565   1.04445952]]


In [8]:
def Click2 (N, eta, p):
# This function calculates the click probability for the simple
# case without taking into account efficiency.
# Arguments: efficiency of the detector, eta; number of photons,
# N, and probability detection, p. In general all will be arrays.

    i = np.arange(len(p))
    eta_new = eta[...,np.newaxis]
    Neta = N * eta_new # First row of this matrix is N multiplied by eta1
                       # Second row is N multiplied by eta2, ...
    Neta_new = Neta[..., np.newaxis]
    addend = Neta_new**i / factorial(i) * (1-p) # We have a 3D array.
    # addend[i,j,k]. Index i is related with eta. Index j is related with N.
    # Index is related with i (iteration number of te series, not confuse with index i)
    
    SUM = np.sum(addend, axis = 2) #The result is a matrix. Each row corresponds
    # with a different efficiency. Each column correspond to a different number N
    # of photons
    
    return (1 - np.exp(-Neta) * SUM) # Same as in SUM


## Testing the functions
Now we're going to do some tests to check if the results of the functions are consistent.

In [9]:
N = np.array([1, 2, 3, 4, 5, 6]) #This will be the number of photons
eta = np.arange(2) /1 #This all the possible efficiencies
p = np.arange(6)/5 

In [10]:
Click1 (N, p)-Click2 (N, np.array([1]), p)

array([[ 0.,  0.,  0.,  0.,  0.,  0.]])

For the case $\eta = 1$ both functions give the same result

# Click probabilities with a loop

Here we calculate also the click probability but now using loops, which probabily is faster.

In [11]:

def Click3(N, eta, p):
        i = 0
        summ = 0
        j = 0
        R = np.zeros((len(eta),len(N)))
        while (j< len(eta)):
            while (i < len(p)):
                summ = summ + (1-p[i]) * (eta[j]*N)**i / factorial(i)
                i=i+1
            
            R[j,:] = 1-np.exp(-eta[j]*N)*summ
            j = j+1
        return R

First we want to check that we obtin the same results with Click3 and Click2

In [12]:
N = np.linspace(0,10,10) #This will be the number of photons
eta = np.arange(2) /1 #This all the possible efficiencies
p = np.arange(6) /5

print(Click3(N,eta,p)-Click2(N,eta,p))




[[ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.44882503  0.45450374  0.33882953  0.21826455  0.12785839
   0.06984298  0.0361268   0.01788199  0.00853519]]


The result shows that for efficiency 1 the results are the same but for different efficiency the result changes, therefore there is an error.

In [ ]:
#ticB = time.clock()
#Click3(N,eta,p)  #Fastest
#tocB = time.clock()


#tic = time.clock()
#Diif = Click2(N,eta,p)-Click3(N,eta,p)
#toc = time.clock()

#B = tocB-ticB
#print(B)
#C = toc-tic
#print(C)
#B-C

#print(Click2(N,eta,p))


#print(Click3(N,np.array([1]),p)-Click1(N,p))
#print(Click2(N,np.array([1]),p)-Click1(N,p))